### <center>2018 Winter CS101.05</center>

# <center>基于数组的序列</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 课后作业-01

- 今日截止
- 习题讲解

## 今日议程

1. Python的序列类型
2. 底层数组结构
3. 动态数组和摊销
4. Python序列类型的效率
5. 多维数组

## Python的序列类型

Python内置了各种“序列”类，这些类底层都使用了类似数组的结构，它们都支持下标访问，如seq[k]：

- 列表(list)
- 元祖(tuple)
- 字符串(str)




本课程重点介绍序列的：

- 行为
- 实现细节
- 效率分析

## 底层数组结构

- 1byte = 8bit
- RAM(Ramdom Access Memory)
- 内存地址
- 数组：存储在计算机储存器内的一块连续的变量

![](./img/5-1.png)

# Any Questions?